## AutoGen Tooling: Travel Planner Example

*[Coding along with the Udemy online course [Mastering AutoGen: Building Multi-Agent Systems](https://www.udemy.com/course/autogen-agent-systems/?couponCode=MTST7102224A2) by Paulo Dichone]*

AutoGen 2.0 for Python documentation @ https://microsoft.github.io/autogen/0.2/docs/Getting-Started

In [9]:
# https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/conversable_agent/
from autogen import ConversableAgent, UserProxyAgent, AssistantAgent
import pandas as pd

In [2]:
from typing import Annotated

Importing `autogen` returned the following error:<br/>
`flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.`

The temporary solution to solve this was to create a constraints.txt file with the following content:

`flaml==2.2.0`

Then running: `pip install -c constraints.txt autogen`

Following the suggestion at https://github.com/microsoft/autogen/issues/3548.

*FLAML @ https://microsoft.github.io/FLAML/ is "A Fast Library for Automated Machine Learning & Tuning."*

In [3]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [4]:
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0.0,
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


#### __Define travel planner functions__

In [7]:
def calculate_travel_time(
    distance: Annotated[int, "Distance in kilometers"],
    speed: Annotated[int, "Speed in km/h"],
) -> str:
    travel_time = distance / speed
    return f"At a speed of {speed} km/h, it will take approximately {travel_time:.2f} hours to travel {distance} kilometers."


def convert_currency(
    amount: Annotated[float, "Amount in USD"],
    rate: Annotated[float, "Exchange rate to EUR"],
) -> str:
    converted_amount = amount * rate
    return f"${amount} USD is approximately €{converted_amount:.2f} EUR."


def suggest_activity(location: Annotated[str, "Location"]) -> str:
    activities = {
        "Paris": "Visit the Eiffel Tower and the Louvre Museum.",
        "New York": "See Times Square and Central Park.",
        "Tokyo": "Explore the Shibuya Crossing and the Senso-ji Temple.",
    }
    return activities.get(location, f"No specific activities found for {location}.")

#### __Define the assistant agent that suggests tool calls__

In [10]:
assistant = AssistantAgent(
    name="TravelPlannerAssistant",
    system_message="You are a helpful AI travel planner. Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)
assistant

#### __Define the user proxy agent__

In [13]:
user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    code_execution_config = {
        "use_docker": False
    },
    human_input_mode="TERMINATE",
)
user_proxy

#### __Register the tool signatures with the assistant agent__

In [14]:
assistant.register_for_llm(
    name="calculate_travel_time",
    description="Calculate travel time based on distance and speed",
)(calculate_travel_time)

assistant.register_for_llm(
    name="convert_currency", description="Convert USD to EUR based on exchange rate"
)(convert_currency)

assistant.register_for_llm(
    name="suggest_activity", description="Suggest activities for a specific location"
)(suggest_activity)

<function __main__.suggest_activity(location: Annotated[str, 'Location']) -> str>

#### __Registering the tool functions with the user proxy agent__

In [15]:
user_proxy.register_for_execution(name="calculate_travel_time")(calculate_travel_time)

user_proxy.register_for_execution(name="convert_currency")(convert_currency)

user_proxy.register_for_execution(name="suggest_activity")(suggest_activity)

<function __main__.suggest_activity(location: Annotated[str, 'Location']) -> str>

#### __Initiate the chat__

In [17]:
user_proxy.initiate_chat(
    assistant, message="I am planning a trip to Paris. What should I do there?"
)

User (to TravelPlannerAssistant):

I am planning a trip to Paris. What should I do there?

--------------------------------------------------------------------------------
TravelPlannerAssistant (to User):

***** Suggested tool call (call_7qkX5mBwnW056jdS4Pzuvzga): suggest_activity *****
Arguments: 
{"location":"Paris"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION suggest_activity...
User (to TravelPlannerAssistant):

User (to TravelPlannerAssistant):

***** Response from calling tool (call_7qkX5mBwnW056jdS4Pzuvzga) *****
Visit the Eiffel Tower and the Louvre Museum.
**********************************************************************

--------------------------------------------------------------------------------
TravelPlannerAssistant (to User):

In Paris, you should definitely visit the Eiffel Tower and t

Please give feedback to TravelPlannerAssistant. Press enter or type 'exit' to stop the conversation:  what can you suggest in brazil?


User (to TravelPlannerAssistant):

what can you suggest in brazil?

--------------------------------------------------------------------------------
TravelPlannerAssistant (to User):

***** Suggested tool call (call_y1pcz1sBsrktpGgzKQIHbYcE): suggest_activity *****
Arguments: 
{"location":"Brazil"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION suggest_activity...
User (to TravelPlannerAssistant):

User (to TravelPlannerAssistant):

***** Response from calling tool (call_y1pcz1sBsrktpGgzKQIHbYcE) *****
No specific activities found for Brazil.
**********************************************************************

--------------------------------------------------------------------------------
TravelPlannerAssistant (to User):

It seems I couldn't find specific activities for Brazil. However, Brazil is known for i

Please give feedback to TravelPlannerAssistant. Press enter or type 'exit' to stop the conversation:  what about tangier?


User (to TravelPlannerAssistant):

what about tangier?

--------------------------------------------------------------------------------
TravelPlannerAssistant (to User):

***** Suggested tool call (call_V47NTf8onPsrZVv9YTKN0hmY): suggest_activity *****
Arguments: 
{"location":"Tangier"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION suggest_activity...
User (to TravelPlannerAssistant):

User (to TravelPlannerAssistant):

***** Response from calling tool (call_V47NTf8onPsrZVv9YTKN0hmY) *****
No specific activities found for Tangier.
**********************************************************************

--------------------------------------------------------------------------------
TravelPlannerAssistant (to User):

It seems I couldn't find specific activities for Tangier. However, here are some popular things t

Please give feedback to TravelPlannerAssistant. Press enter or type 'exit' to stop the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': 'I am planning a trip to Paris. What should I do there?', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_7qkX5mBwnW056jdS4Pzuvzga', 'function': {'arguments': '{"location":"Paris"}', 'name': 'suggest_activity'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'Visit the Eiffel Tower and the Louvre Museum.', 'tool_responses': [{'tool_call_id': 'call_7qkX5mBwnW056jdS4Pzuvzga', 'role': 'tool', 'content': 'Visit the Eiffel Tower and the Louvre Museum.'}], 'role': 'tool', 'name': 'User'}, {'content': 'In Paris, you should definitely visit the Eiffel Tower and the Louvre Museum. Enjoy your trip! \n\nTERMINATE', 'role': 'user', 'name': 'TravelPlannerAssistant'}, {'content': 'what can you suggest in brazil?', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_y1pcz1sBsrktpGgzKQIHbYcE', 'function': {'arguments': '{"location":"Brazil"}', 'name': 'suggest_activity'}, 'type': 'function'}]